In [1]:
import os
import pandas as pd

version = "v01"
main_publisher = 'MIT'
script_dir = os.path.dirname(os.path.realpath('__file__'))
path = os.path.join(script_dir, '../Data/' + main_publisher + '/')
df_chapters = pd.read_csv(path + 'chapters.csv', sep = '|')
df_chapters.head()

,Cid,Eid,Sid,Title,Text,BeginTimestamp,EndTimestamp,Corpus
0,0,0,0,"Lecture 1: Motivation, Intuition, and Examples",[SQUEAKING] [RUSTLING] [CLICKING] PAIGE BRIGHT...,0:00:00,0:59:33,MIT OpenCourseWare
1,1,1,0,Lecture 2: General Theory Part 0,[SQUEAKING] [RUSTLING] [CLICKING] PAIGE BRIGHT...,0:00:00,0:21:17,MIT OpenCourseWare
2,2,1,0,Lecture 2: General Theory Part 1,"I know that most of them-- in fact, infinitely...",0:21:44,0:43:37,MIT OpenCourseWare
3,3,1,0,Lecture 2: General Theory Part 2,You can replace R with just any metric space. ...,0:43:40,1:18:24,MIT OpenCourseWare
4,4,2,0,Lecture 3: Compact Sets in Rⁿ Part 0,[SQUEAKING] [RUSTLING] [CLICKING] PAIGE BRIGHT...,0:00:00,0:21:20,MIT OpenCourseWare


In [2]:
len(df_chapters)

1860

In [3]:
df_chapters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860 entries, 0 to 1859
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Cid             1860 non-null   int64 
 1   Eid             1860 non-null   int64 
 2   Sid             1860 non-null   int64 
 3   Title           1860 non-null   object
 4   Text            1857 non-null   object
 5   BeginTimestamp  1860 non-null   object
 6   EndTimestamp    1860 non-null   object
 7   Corpus          1860 non-null   object
dtypes: int64(3), object(5)
memory usage: 116.4+ KB


In [5]:
from concepts import text2RDF

df_chapters.dropna().apply(lambda x : text2RDF(x['Text'], "../Output/Graphs/"+ version + "/"+ main_publisher + "/" + str(x['Cid'])), axis = 1)

Wikification for : 0
Graph generation for : 0
Saved in  ../Output/Graphs/v01/MIT/0.ttl
Wikification for : 1
Graph generation for : 1
Saved in  ../Output/Graphs/v01/MIT/1.ttl
Wikification for : 2
Graph generation for : 2
Saved in  ../Output/Graphs/v01/MIT/2.ttl
Wikification for : 3
Graph generation for : 3
Saved in  ../Output/Graphs/v01/MIT/3.ttl
Wikification for : 4
Graph generation for : 4
Saved in  ../Output/Graphs/v01/MIT/4.ttl
Wikification for : 5
Graph generation for : 5
Saved in  ../Output/Graphs/v01/MIT/5.ttl
Wikification for : 6
Graph generation for : 6
Saved in  ../Output/Graphs/v01/MIT/6.ttl
Wikification for : 7
Graph generation for : 7
Saved in  ../Output/Graphs/v01/MIT/7.ttl
Wikification for : 8
Graph generation for : 8
Saved in  ../Output/Graphs/v01/MIT/8.ttl
Wikification for : 9
Graph generation for : 9
Saved in  ../Output/Graphs/v01/MIT/9.ttl
Wikification for : 10
Graph generation for : 10
Saved in  ../Output/Graphs/v01/MIT/10.ttl
Wikification for : 11
Graph generation f

0       ((http://en.wikipedia.org/wiki/Direct_democrac...
1       ((http://example.org/Chapter, http://purl.org/...
2       ((http://en.wikipedia.org/wiki/Utilitarianism,...
3       ((http://en.wikipedia.org/wiki/Direct_democrac...
4       ((http://en.wikipedia.org/wiki/Smallest_organi...
                              ...                        
1855    ((http://en.wikipedia.org/wiki/Mean, http://pu...
1856    ((http://en.wikipedia.org/wiki/History_of_tele...
1857    ((http://en.wikipedia.org/wiki/Gang, https://u...
1858    ((http://en.wikipedia.org/wiki/History_of_tele...
1859    ((http://en.wikipedia.org/wiki/Mean, http://pu...
Length: 1857, dtype: object

In [6]:
from rdflib import Graph

chapters = df_chapters['Cid'].values
df_concepts = pd.DataFrame()
missing = []
for c in chapters:
    concepts = []
    path_graph = f'../Output/Graphs/{version}/{main_publisher}/{c}.ttl'
    g = Graph()
    try :
        g.parse(path_graph, format='turtle')
    except (FileNotFoundError, IOError):
        print(f'Chapter {c} file not found !')
        missing.append(c)
        continue
    concepts_query = """
            SELECT ?concept ?pr WHERE
            {
                ?ER dct:subject ?concept.
                ?concept ns1:pageRank ?pr.
            }
    """
    result = g.query(concepts_query)
    for row in result:
        concept_dict = {
            'OER' : c,
            'Concept' : str(row.concept),
            'PR' : float(str(row.pr))
        }
        concepts.append(concept_dict)
    df_concepts = pd.concat([df_concepts, pd.DataFrame(concepts)], ignore_index = True)
print(df_concepts.shape)
df_concepts.to_csv(path + 'concepts.csv', sep = '|', index = False)

Chapter 406 file not found !
Chapter 440 file not found !
Chapter 720 file not found !
(1034068, 3)


In [7]:
pr_filter = 0.005
df_concepts_bis = df_concepts[df_concepts['PR'] > pr_filter]

print(len(df_concepts), '\tNUM of concepts')
print(len(df_concepts_bis), '\tNUM of concepts w filter on PR = ',pr_filter)
print(round(len(df_concepts)/len(df_concepts_bis), 2), "\t% Compression after filtering")

value_counts = df_concepts_bis['OER'].value_counts()
rec_value_mean = int(value_counts.mean())
print(rec_value_mean, '\tAVG concepts per chapter')

rec_value_median = int(value_counts.median())
print(rec_value_median, '\tMEDIAN concepts per chapter')

rec_value_max = value_counts.max()
print(rec_value_max, '\tMAX concepts per chapter')

rec_value_min = value_counts.min()
print(rec_value_min, '\tMIN concepts per chapter')

df_concepts_bis.to_csv(path + 'concepts_bis.csv', sep = '|', index = False)

1034068 	NUM of concepts
31681 	NUM of concepts w filter on PR =  0.005
32.64 	% Compression after filtering
17 	AVG concepts per chapter
14 	MEDIAN concepts per chapter
54 	MAX concepts per chapter
2 	MIN concepts per chapter
